In [2]:
# sentiment analysis RNN

In [3]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM, GRU, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import imdb
import numpy as np

In [5]:
# getting reviews with words that come under 5000
# most occuring words int the entire
# corps of textual review data

vocab_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

print(X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]


In [10]:
# getting all the words from word_index dictionary
word_idx = imdb.get_word_index()

# originally the index number of a value and not a key
# hence converting the index as a key and the words as avalues
word_idx = {i:word for word,i in word_idx.items()}

# again printing the reviews
print([word_idx[i] for i in X_train[0]])

['the', 'as', 'you', 'with', 'out', 'themselves', 'powerful', 'lets', 'loves', 'their', 'becomes', 'reaching', 'had', 'journalist', 'of', 'lot', 'from', 'anyone', 'to', 'have', 'after', 'out', 'atmosphere', 'never', 'more', 'room', 'and', 'it', 'so', 'heart', 'shows', 'to', 'years', 'of', 'every', 'never', 'going', 'and', 'help', 'moments', 'or', 'of', 'every', 'chest', 'visual', 'movie', 'except', 'her', 'was', 'several', 'of', 'enough', 'more', 'with', 'is', 'now', 'current', 'film', 'as', 'you', 'of', 'mine', 'potentially', 'unfortunately', 'of', 'you', 'than', 'him', 'that', 'with', 'out', 'themselves', 'her', 'get', 'for', 'was', 'camp', 'of', 'you', 'movie', 'sometimes', 'movie', 'that', 'with', 'scary', 'but', 'and', 'to', 'story', 'wonderful', 'that', 'in', 'seeing', 'in', 'character', 'to', 'of', '70s', 'and', 'with', 'heart', 'had', 'shadows', 'they', 'of', 'here', 'that', 'with', 'her', 'serious', 'to', 'have', 'does', 'when', 'from', 'why', 'what', 'have', 'critics', 'they'

In [9]:
# get the minimum and maimum length of the reviews
print("Max length of a review:: ", len(max(X_train+X_test, key=len)))
print("Min length of a review:: ", len(min(X_train+X_test, key=len)))

Max length of a review::  2697
Min length of a review::  70


In [12]:
from tensorflow.keras.preprocessing import sequence

# keeping a fixed length of all reviews
max_len =400

X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)

X_valid, y_valid = X_train[:64], y_train[:64]
X_train, y_train = X_train[64:], y_train[64:]

In [15]:
# fixing every word's embedding size to be 32
embed_size = 32

# creating a RNN model
RNN_model = Sequential(name="Simple_RNN")
RNN_model.add(Embedding(vocab_size,
                        embed_size,
                        input_length=(max_len)))

# in case of a stacked(more than one layer of RNN)
# use return_sequential=True
RNN_model.add(SimpleRNN(32,
                        activation='tanh',
                        return_sequences=False))
RNN_model.add(Dense(1, activation='sigmoid'))

# printing model summary
print(RNN_model.summary())

# compeling model
RNN_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

# fitting model # training the model
history = RNN_model.fit(X_train, y_train,
              batch_size=64,
              epochs=5,
              verbose=1,
              validation_data=(X_valid, y_valid))

print()
print("Simple_RNN Score ---> ", RNN_model.evaluate(X_test, y_test, verbose=0))

Model: "Simple_RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 400, 32)           160000    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 162113 (633.25 KB)
Trainable params: 162113 (633.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/5
390/390 [==============================] - 46s 115ms/step - loss: 0.5866 - accuracy: 0.6845 - val_loss: 0.4606 - val_accuracy: 0.8281
Epoch 2/5
390/390 [==============================] - 43s 110ms/step - loss: 0.4091 - accuracy: 0.8227 - val_loss: 0.3833 - val_accu

In [17]:
# Defining GRU model
gru_model = Sequential(name="GRU_Model")
gru_model.add(Embedding(vocab_size,
                        embed_size,
                        input_length=(max_len)))

gru_model.add(GRU(32,
                  activation='tanh',
                  return_sequences=False))
gru_model.add(Dense(1, activation='sigmoid'))

# printing model summary
print(gru_model.summary())

gru_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

# fitting model # training the GRU model
history = gru_model.fit(X_train, y_train,
              batch_size=64,
              epochs=5,
              verbose=1,
              validation_data=(X_valid, y_valid))

print()
print("GRU Score ---> ", gru_model.evaluate(X_test, y_test, verbose=0))

Model: "GRU_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 400, 32)           160000    
                                                                 
 gru_1 (GRU)                 (None, 32)                6336      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 166369 (649.88 KB)
Trainable params: 166369 (649.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/5
390/390 [==============================] - 84s 208ms/step - loss: 0.4420 - accuracy: 0.7810 - val_loss: 0.2449 - val_accuracy: 0.9062
Epoch 2/5
390/390 [==============================] - 82s 211ms/step - loss: 0.2724 - accuracy: 0.8927 - val_loss: 0.2861 - val_accur

Bidirectional LSTMS are a derivative of a treaditional LSTMS. Here, two LSEMs are used to capture the forward and backward sequences of the input. This helps in capturing the context better than normal LSTM.

In [ ]:
# Defining LSTM model
lstm_model = Sequential(name="LSTM_Model")
lstm_model.add(Embedding(vocab_size,
                        embed_size,
                        input_length=(max_len)))
lstm_model.add(LSTM(128,
                  activation='tanh',
                  return_sequences=False))
lstm_model.add(Dense(1, activation='sigmoid'))

# printing model summary
print(lstm_model.summary())

#compile the model
lstm_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

# fitting model # training the LSTM
history3 = lstm_model.fit(X_train, y_train,
              batch_size=64,
              epochs=5,
              verbose=1,
              validation_data=(X_valid, y_valid))

print()
print("LSTM Score ---> ", lstm_model.evaluate(X_test, y_test, verbose=0))

Model: "LSTM_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 400, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 128)               82432     
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 242561 (947.50 KB)
Trainable params: 242561 (947.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/5
390/390 [==============================] - 314s 797ms/step - loss: 0.4932 - accuracy: 0.7512 - val_loss: 0.2917 - val_accuracy: 0.9062
Epoch 2/5
390/390 [==============================] - 307s 787ms/step - loss: 0.3083 - accuracy: 0.8752 - val_loss: 0.2473 - val_ac

In [ ]:
# Defining Bidirectional LSTM Model
bi_lstm_model = Sequential(name="Bi_LSTM_Model")
bi_lstm_model.add(Embedding(vocab_size,
                        embed_size,
                        input_length=(max_len)))
bi_lstm_model.add(Bidirectional(LSTM(128,
                  activation='tanh',
                  input_length=(max_len))))

bi_lstm_model.add(Dense(1, activation='sigmoid'))

# printing model summary
print(bi_lstm_model.summary())

#compile the model
bi_lstm_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

# fitting model # training the LSTM
history4 = bi_lstm_model.fit(X_train, y_train,
              batch_size=64,
              epochs=5,
              verbose=1,
              validation_data=(X_valid, y_valid))

print()
print("LSTM Score ---> ",bi_lstm_model.evaluate(X_test, y_test, verbose=0))